In [1]:
import os
import pandas as pd
import numpy as np
import yaml


quantiles_thresholds = [0.5]#[0.33, 0.67]

mm_ratings_df = pd.read_csv("mm_ratings.csv")
mf_ratings_df = pd.read_csv("mf_ratings.csv")
groundtruth_df = pd.read_csv("../files/groundtruth.csv", index_col="stimulus_id")

for ratings_df in [mm_ratings_df, mf_ratings_df]:
    ratings_df = ratings_df.drop(["prolific_id"], axis=1)
    ratings_means_df = ratings_df.groupby('stimulus_id').mean()
    # merge with groundtruth
    groundtruth_df = pd.merge(groundtruth_df, ratings_means_df, left_index=True, right_index=True)

# drop these columns
drop_these = [ 
    'asian','black', 'white', 'other', 'title', 'description', 'upload_date',
    'duration', 'view_count', 'categories', 'tags', 'like_count','requested_subtitles', 
    'download', 'error_logs',
]

groundtruth_df = groundtruth_df.drop(drop_these, axis=1)

# bin the values to 3 categories
for column in groundtruth_df.columns:
    if groundtruth_df[column].dtype == 'float64':
        quantiles = groundtruth_df[column].quantile(quantiles_thresholds).tolist() 
        groundtruth_df[column] = pd.cut(groundtruth_df[column], bins=[-np.inf, *quantiles, np.inf], labels=[0, 1])
        groundtruth_df[column] = groundtruth_df[column].astype('int64')

groundtruth_df.head()

# save the groundtruth
groundtruth_df.to_csv('music_groundtruth.csv')

# ROUND 2 (continue)
with open('config_binary_training.yaml', 'r') as f:
    config = yaml.safe_load(f)

fold_music_df = pd.read_csv('music_groundtruth.csv', index_col="stimulus_id")

emo_to_label = {
    0: "low",
    1: "high",
    #2: "very"
}

for emotion in config["emo_dict"].keys():
    fold_music_df[emotion] = fold_music_df[emotion].map(emo_to_label)

for mid in config["mid_dict"].keys():
    fold_music_df[mid][fold_music_df[mid] == 0] = mid.split("/")[0]
    #fold_music_df[mid][fold_music_df[mid] == 1] = "undefined"
    fold_music_df[mid][fold_music_df[mid] == 1] = mid.split("/")[1]
    fold_music_df[mid] = fold_music_df[mid].astype(str)

cols = (
    list(config["emo_dict"].keys()) + list(config["mid_dict"].keys()) + list(config["cls_dict"]) + ["all_genders"]
)

cols.remove("target_of_toy_ad")

fold_music_df = fold_music_df[cols]

# rename column "all_genders" to "target_of_toy_ad"
fold_music_df = fold_music_df.rename(columns={"all_genders": "target_of_toy_ad"})

fold_music_df.to_csv('music_binary_groundtruth.csv')

/tmp/ipykernel_3317766/692043103.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_music_df[mid][fold_music_df[mid] == 0] = mid.split("/")[0]
/tmp/ipykernel_3317766/692043103.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_music_df[mid][fold_music_df[mid] == 0] = mid.split("/")[0]
/tmp/ipykernel_3317766/692043103.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_music_df[mid][fold_music_df[mid] == 0] = mid.spl

In [2]:
print(fold_music_df.columns)
display(fold_music_df.head())

Index(['Happy', 'Amusing', 'Beauty', 'Calm', 'Energizing', 'Angry',
       'Triumphant', 'Many Instruments/Few Instruments',
       'Wide pitch variation/Narrow pitch variation', 'Clear melody/No melody',
       'Repetitive/Non-repetitive', 'Complex rhythm/Simple rhythm',
       'Fast tempo/Slow tempo', 'Strong beat/Weak beat', 'Electric/Acoustic',
       'Distorted/Clear', 'Loud/Soft', 'Heavy/Light', 'High pitch/Low pitch',
       'Punchy/Smooth', 'Harmonious/Disharmonious', 'Dense/Sparse',
       'voice_age', 'voice_gender', 'voice_exagg', 'voice_type',
       'target_of_toy_ad'],
      dtype='object')


,Happy,Amusing,Beauty,Calm,Energizing,Angry,Triumphant,Many Instruments/Few Instruments,Wide pitch variation/Narrow pitch variation,Clear melody/No melody,...,Heavy/Light,High pitch/Low pitch,Punchy/Smooth,Harmonious/Disharmonious,Dense/Sparse,voice_age,voice_gender,voice_exagg,voice_type,target_of_toy_ad
stimulus_id,,,,,,,,,,,,,,,,,,,,,
ndzo2ZIWfiQ,high,low,high,high,high,high,high,Many Instruments,Narrow pitch variation,Clear melody,...,Light,High pitch,Smooth,Harmonious,Dense,Adults (including young adults),Feminine,"No, all voices are normal-sounding",BOTH spoken and sung,Girls/women
yRUiwRKk6QM,low,high,low,low,high,high,high,Many Instruments,Wide pitch variation,No melody,...,Heavy,Low pitch,Punchy,Disharmonious,Dense,Adults (including young adults),Masculine,Yes a masculine voice is gender exaggerated,Spoken,Mixed
3ysC1-foJT4,high,low,high,high,high,low,low,Few Instruments,Wide pitch variation,Clear melody,...,Heavy,Low pitch,Punchy,Disharmonious,Dense,Adults (including young adults),Feminine,Yes a feminine voice is gender exaggerated,Sung,Girls/women
cYszuGaptkk,low,low,low,low,high,high,high,Many Instruments,Wide pitch variation,No melody,...,Heavy,Low pitch,Smooth,Disharmonious,Sparse,Adults (including young adults),Feminine,"No, all voices are normal-sounding",Spoken,Mixed
2LZjLBipdfI,high,high,high,high,low,low,low,Few Instruments,Wide pitch variation,Clear melody,...,Light,High pitch,Smooth,Disharmonious,Sparse,Adults (including young adults),Feminine,Yes a feminine voice is gender exaggerated,BOTH spoken and sung,Girls/women
